# Rap Music Analysis

In [19]:
# ! pip install requests
# ! pip install beautifulsoup4
# ! pip install selenium

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# driver = webdriver.Firefox()

In [6]:
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen
import requests

In [7]:
#API for Genius

ClientID = 'YKXpjqrhn6TBeUGu_Cl2R5WxX1n9ocoyVp1BCVSD0wB24P9AiZfRpEx9yosghZRZ'
Client_secret = 'LbvQg5Qr4Z15ccjx5mIDRqoIUOW-5rhfQmeCuSjgLujXXKA7JSUc0RUl2aAhppdTATjE4MOK2vK3vFHZlSjxJA'


In [8]:
#search for song

client_access_token = 'XCXzd1AHCXzteLWAlqqSkZElvC4YGsux7DECb89Bbour93_g80bNgMScWVVNoyuK'
base_url = 'https://api.genius.com'

user_input = input('artist and song: ').replace(" ", "-")

path = 'search/'
request_uri = '/'.join([base_url, path])
print(request_uri + user_input)

params = {'q': user_input}

token = 'Bearer {}'.format(client_access_token)
headers = {'Authorization': token}

r = requests.get(request_uri, params=params, headers=headers)
print(r.text)

artist and song: Rihana
https://api.genius.com/search/Rihana
{"meta":{"status":200},"response":{"hits":[{"highlights":[],"index":"song","type":"song","result":{"annotation_count":1,"api_path":"/songs/2165535","full_title":"Fuck boy by Rihana abdulkaf","header_image_thumbnail_url":"https://assets.genius.com/images/default_cover_image.png?1594314430","header_image_url":"https://assets.genius.com/images/default_cover_image.png?1594314430","id":2165535,"lyrics_owner_id":1915487,"lyrics_state":"complete","path":"/Rihana-abdulkaf-fuck-boy-lyrics","pyongs_count":null,"song_art_image_thumbnail_url":"https://assets.genius.com/images/default_cover_image.png?1594314430","song_art_image_url":"https://assets.genius.com/images/default_cover_image.png?1594314430","stats":{"unreviewed_annotations":1,"hot":false},"title":"Fuck boy","title_with_featured":"Fuck boy","url":"https://genius.com/Rihana-abdulkaf-fuck-boy-lyrics","primary_artist":{"api_path":"/artists/514599","header_image_url":"https://assets

In [25]:
ctx = ssl.create_default_context()
ctx.check_hostname= False
ctx.verify_mode = ssl.CERT_NONE

In [15]:
req = Request(url, headers = {"User-Agent" : "Mozilla/5.0"})
webpage = urlopen(req).read()

In [18]:
urls = ['https://genius.com/Eminem-rap-god-lyrics',
        'https://genius.com/Kendrick-lamar-humble-lyrics'] # populate with links to song lyrics

for url in urls: 

    # Create a BeautifulSoup object 
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    song_json={}
    song_json['Lyrics'] = [];
    song_json['Comments']=[];


    # Extract title of the song
    for title in soup.findAll('title'): 
        song_json['Title']=title.text.strip()

    # Extract artist of the song
    for artist in soup.findAll('Primary Artist'): 
        song_json['Artist']=artist.text.strip()

    # Extract user comments on the song
    for div in soup.findAll('div',attrs={'class':'rich_text_formatting'}):
        comments = div.text.strip().split("\n")

        for comment in comments: 
            if comment != "":
                song_json['Comments'].append(comment);

    # Extract the lyrics of the song
    for div in soup.findAll('div', attrs = {'class':'lyrics'}):
        song_json['Lyrics'].append(div.text.strip().split("\n")); 

    #Save the json created with the file name as title + .json
    with open(song_json["Title"] + '.json', 'w') as outfile:
        json.dump(song_json, outfile, indent = 4, ensure_ascii = False)

    # Save the html content into an html file with name as title + .html
    with open(song_json["Title"] + '.html', 'wb') as file:
        file.write(html)